In [1]:
#import Files 
import xml.sax
import bz2
import sys
from collections import defaultdict
from titleProcessor import TitleProcessor
from TextParser import TextFieldProcessor
from PageProcessor import PageProcessor
import re
import timeit


[nltk_data] Downloading package stopwords to /home/apurvi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /home/apurvi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#global Variables 
Doc_ID=1 #to maintain total Number of pages in the dump
Doc_IDtoTitle={} # a map to maintain name of doc ids
globalDictionary={} #final invertedIndex dictionary for each batch
invertedIndexFile=1 #maintainsFile number

In [3]:
def getGlobalDictionary(pageDictionary):
    global globalDictionary,Doc_ID
    for word in pageDictionary:
        check=checkWords(word)
        if check==True:
            continue
        pageDict={}
        if word in globalDictionary :
            pageDict=globalDictionary[word]
        pageDict[Doc_ID]=pageDictionary[word]
        globalDictionary[word]=pageDict

def writeToFile():
    global globalDictionary,invertedIndexFile,Doc_ID
    outputPath="./output"
    filePath=outputPath+"/indexfile"+str(invertedIndexFile)+".txt"
    invertedIndexFile+=1
    with open(filePath, "a") as outfile:
        for word in sorted(globalDictionary):
            invStr=word+":"
#             print("in write file ",invStr)
            tempDict=globalDictionary[word]
            for item in sorted(tempDict):
                invStr+="d"+str(item)
    #             print("add docid",invStr)
                fieldFreq=tempDict[item]
                tagDict={0:"t",1:"i",2:"b",3:"c",4:"l",5:"r"}
                for i in range(0,len(fieldFreq)):
    #                 print("i is",i,fieldFreq[i])
                    if fieldFreq[i]>0:
                        invStr+="#"+tagDict[i]+str(fieldFreq[i])
                invStr+="|"        
            invStr=invStr[:-1]
#             print("final invStr",invStr)
            outfile.write(invStr + '\n')        
    outfile.close()        

In [4]:
def checkWords(word):
    ignoreWords=['redirect','redirect2']
    if len(word)<=2 or word in ignoreWords or re.match('^[0]+$',word) or not(re.match('^[a-zA-Z0-9]+$',word)):
        return True
    return False

In [5]:
#content Handler code for XML parser
def storeValue(key,value,Dict):
    global Doc_IDtoTitle
    Doc_IDtoTitle[key]=value
    
class parseXML(xml.sax.ContentHandler):
    
    global Doc_ID,Doc_IDtoTitle 
    def __init__(self):
        xml.sax.ContentHandler.__init__(self)
        self.current_tag=""
        self.title_content=""
        self.text_content=""
    
    def startElement(self,tag,attributes):
        self.current_tag=tag
        if tag=='page':
            self.title_content=""
            self.text_content=""

    def characters(self,content):
        if self.current_tag=="title":
            self.title_content+=content
        elif self.current_tag=="text":
            self.text_content+=content
            
    def endElement(self,tag):
        if tag=='page':
            global Doc_ID,globalDictionary,Doc_IDtoTitle
            print("----------------------end doc",Doc_ID,"----------------------------")
            if Doc_ID%50000==0:
                writeToFile()
                globalDictionary.clear()
                Doc_IDtoTitle.clear()
            
           
            Doc_ID+=1
        
        elif tag=='title':
#             print(self.title_content)
#             global Doc_ID,globalDictionary,Doc_IDtoTitle
            storeValue(Doc_ID,self.title_content,Doc_IDtoTitle) 
        
        elif tag=='text':
            #process title
            pr=TitleProcessor()
            processedTitle=pr.processData(self.title_content)
#             print("Processed title : ",processedTitle)
            
            #get fields from text_content
            pt=TextFieldProcessor()
            info,bodyText,category,externalLinks,references=pt.process(self.text_content)
            
            #make dictionary for each field
            titleDict=pr.getTitleDictionary(processedTitle)
            page=PageProcessor()
            infoDict=page.getAllDictionary(info)
#             print("---------info---------")
            bodyDict=page.getAllDictionary(bodyText)
#             print("---------bodyDict---------")
            categoryDict=page.getAllDictionary(category)
#             print("---------catDict---------")
            externalLinksDict=page.getAllDictionary(externalLinks)
#             print("---------exlDict---------")
#             print(referencesDict)
            referencesDict=page.getAllDictionary(references)
#             print("---------refDict---------")
            
#             print(externalLinksDict)
            
            #make final page Dictionary for each page
            pageDictionary=page.getPageDictionary(titleDict,infoDict,bodyDict,categoryDict,externalLinksDict,referencesDict)
#             print("page dictionary ",pageDictionary)
            getGlobalDictionary(pageDictionary)
        self.current_tag=""

In [ ]:
if __name__=="__main__":
    global globalDictionary
    Wiki_input_file="./Data/input.bz2"
    source = bz2.BZ2File(Wiki_input_file, "rb")
    start = timeit.default_timer()
    #Create a parser Object
    parser= xml.sax.make_parser()
    
    #turnoff namespaces
    parser.setFeature(xml.sax.handler.feature_namespaces, 0)
    
    #callHandler
    handler= parseXML()
    
    #parse data 
    parser.setContentHandler(handler)
    parser.parse(source)
    
    if bool(globalDictionary):
        writeToFile()
    stop = timeit.default_timer()
    print('Time: ', stop - start)
    print('-----------------------------')

----------------------end doc 1 ----------------------------
----------------------end doc 2 ----------------------------
----------------------end doc 3 ----------------------------
----------------------end doc 4 ----------------------------
----------------------end doc 5 ----------------------------
----------------------end doc 6 ----------------------------
----------------------end doc 7 ----------------------------
----------------------end doc 8 ----------------------------
----------------------end doc 9 ----------------------------
----------------------end doc 10 ----------------------------
----------------------end doc 11 ----------------------------
----------------------end doc 12 ----------------------------
----------------------end doc 13 ----------------------------
----------------------end doc 14 ----------------------------
----------------------end doc 15 ----------------------------
----------------------end doc 16 ----------------------------
-----------------

----------------------end doc 134 ----------------------------
----------------------end doc 135 ----------------------------
----------------------end doc 136 ----------------------------
----------------------end doc 137 ----------------------------
----------------------end doc 138 ----------------------------
----------------------end doc 139 ----------------------------
----------------------end doc 140 ----------------------------
----------------------end doc 141 ----------------------------
----------------------end doc 142 ----------------------------
----------------------end doc 143 ----------------------------
----------------------end doc 144 ----------------------------
----------------------end doc 145 ----------------------------
----------------------end doc 146 ----------------------------
----------------------end doc 147 ----------------------------
----------------------end doc 148 ----------------------------
----------------------end doc 149 ---------------------

----------------------end doc 265 ----------------------------
----------------------end doc 266 ----------------------------
----------------------end doc 267 ----------------------------
----------------------end doc 268 ----------------------------
----------------------end doc 269 ----------------------------
----------------------end doc 270 ----------------------------
----------------------end doc 271 ----------------------------
----------------------end doc 272 ----------------------------
----------------------end doc 273 ----------------------------
----------------------end doc 274 ----------------------------
----------------------end doc 275 ----------------------------
----------------------end doc 276 ----------------------------
----------------------end doc 277 ----------------------------
----------------------end doc 278 ----------------------------
----------------------end doc 279 ----------------------------
----------------------end doc 280 ---------------------

----------------------end doc 396 ----------------------------
----------------------end doc 397 ----------------------------
----------------------end doc 398 ----------------------------
----------------------end doc 399 ----------------------------
----------------------end doc 400 ----------------------------
----------------------end doc 401 ----------------------------
----------------------end doc 402 ----------------------------
----------------------end doc 403 ----------------------------
----------------------end doc 404 ----------------------------
----------------------end doc 405 ----------------------------
----------------------end doc 406 ----------------------------
----------------------end doc 407 ----------------------------
----------------------end doc 408 ----------------------------
----------------------end doc 409 ----------------------------
----------------------end doc 410 ----------------------------
----------------------end doc 411 ---------------------

----------------------end doc 529 ----------------------------
----------------------end doc 530 ----------------------------
----------------------end doc 531 ----------------------------
----------------------end doc 532 ----------------------------
----------------------end doc 533 ----------------------------
----------------------end doc 534 ----------------------------
----------------------end doc 535 ----------------------------
----------------------end doc 536 ----------------------------
----------------------end doc 537 ----------------------------
----------------------end doc 538 ----------------------------
----------------------end doc 539 ----------------------------
----------------------end doc 540 ----------------------------
----------------------end doc 541 ----------------------------
----------------------end doc 542 ----------------------------
----------------------end doc 543 ----------------------------
----------------------end doc 544 ---------------------

...

----------------------end doc 2482 ----------------------------
----------------------end doc 2483 ----------------------------
----------------------end doc 2484 ----------------------------
----------------------end doc 2485 ----------------------------
----------------------end doc 2486 ----------------------------
----------------------end doc 2487 ----------------------------
----------------------end doc 2488 ----------------------------
----------------------end doc 2489 ----------------------------
----------------------end doc 2490 ----------------------------
----------------------end doc 2491 ----------------------------
----------------------end doc 2492 ----------------------------
----------------------end doc 2493 ----------------------------
----------------------end doc 2494 ----------------------------
----------------------end doc 2495 ----------------------------
----------------------end doc 2496 ----------------------------
----------------------end doc 2497 -----

----------------------end doc 2611 ----------------------------
----------------------end doc 2612 ----------------------------
----------------------end doc 2613 ----------------------------
----------------------end doc 2614 ----------------------------
----------------------end doc 2615 ----------------------------
----------------------end doc 2616 ----------------------------
----------------------end doc 2617 ----------------------------
----------------------end doc 2618 ----------------------------
----------------------end doc 2619 ----------------------------
----------------------end doc 2620 ----------------------------
----------------------end doc 2621 ----------------------------
----------------------end doc 2622 ----------------------------
----------------------end doc 2623 ----------------------------
----------------------end doc 2624 ----------------------------
----------------------end doc 2625 ----------------------------
----------------------end doc 2626 -----

----------------------end doc 2740 ----------------------------
----------------------end doc 2741 ----------------------------
----------------------end doc 2742 ----------------------------
----------------------end doc 2743 ----------------------------
----------------------end doc 2744 ----------------------------
----------------------end doc 2745 ----------------------------
----------------------end doc 2746 ----------------------------
----------------------end doc 2747 ----------------------------
----------------------end doc 2748 ----------------------------
----------------------end doc 2749 ----------------------------
----------------------end doc 2750 ----------------------------
----------------------end doc 2751 ----------------------------
----------------------end doc 2752 ----------------------------
----------------------end doc 2753 ----------------------------
----------------------end doc 2754 ----------------------------
----------------------end doc 2755 -----

----------------------end doc 2869 ----------------------------
----------------------end doc 2870 ----------------------------
----------------------end doc 2871 ----------------------------
----------------------end doc 2872 ----------------------------
----------------------end doc 2873 ----------------------------
----------------------end doc 2874 ----------------------------
----------------------end doc 2875 ----------------------------
----------------------end doc 2876 ----------------------------
----------------------end doc 2877 ----------------------------
----------------------end doc 2878 ----------------------------
----------------------end doc 2879 ----------------------------
----------------------end doc 2880 ----------------------------
----------------------end doc 2881 ----------------------------
----------------------end doc 2882 ----------------------------
----------------------end doc 2883 ----------------------------
----------------------end doc 2884 -----

----------------------end doc 2999 ----------------------------
----------------------end doc 3000 ----------------------------
----------------------end doc 3001 ----------------------------
----------------------end doc 3002 ----------------------------
----------------------end doc 3003 ----------------------------
----------------------end doc 3004 ----------------------------
----------------------end doc 3005 ----------------------------
----------------------end doc 3006 ----------------------------
----------------------end doc 3007 ----------------------------
----------------------end doc 3008 ----------------------------
----------------------end doc 3009 ----------------------------
----------------------end doc 3010 ----------------------------
----------------------end doc 3011 ----------------------------
----------------------end doc 3012 ----------------------------
----------------------end doc 3013 ----------------------------
----------------------end doc 3014 -----